# Tutorial 6: Classification

### Lecture and Tutorial Learning Goals:

After completing this week's lecture and tutorial work, you will be able to:

* Recognize situations where a simple classifier would be appropriate for making predictions.
* Explain the k-nearest neighbour classification algorithm.
* Interpret the output of a classifier.
* Compute, by hand, the distance between points when there are two explanatory variables/predictors.
* Describe what a training data set is and how it is used in classification.
* In a dataset with two explanatory variables/predictors, perform k-nearest neighbour classification in Python using `scikit-learn` to predict the class of a single new observation.

This tutorial covers parts of [Chapter 5](https://python.datasciencebook.ca/classification1.html) of the online textbook. You should read this chapter before attempting the tutorial. Any place you see `___`, you must fill in the function, variable, or data to complete the code. Substitute the `raise NotImplementedError` with your completed code and answers then proceed to run the cell.

In [ ]:
### Run this cell before continuing.
import random

import altair as alt
import pandas as pd
import numpy as np
from sklearn import set_config
from sklearn.compose import make_column_transformer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler


# Simplify working with large datasets in Altair
alt.data_transformers.disable_max_rows()

# Output dataframes instead of arrays
set_config(transform_output="pandas")

**Question 0.1** Multiple Choice: 
<br> {points: 1}

Before applying k-nearest neighbour to a classification task, we need to scale the data. What is the purpose of this step?

A. To help speed up the knn algorithm. 

B. To convert all data observations to numeric values. 

C. To ensure all data observations will be on a comparable scale and contribute equal shares to the calculation of the distance between points.

D. None of the above. 

*Assign your answer to an object called `answer0_1`. Make sure the correct answer is an uppercase letter. Surround your answer with quotation marks (e.g. `"F"`).*

*Note: we typically **standardize** (i.e., scale **and** center) the data before doing classification. For the K-nearest neighbour algorithm specifically, centering has no effect. But it doesn't hurt, and can help with other predictive data analyses, so we will do it below to start forming a good habit.*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer0_1)).encode("utf-8")+b"59e8").hexdigest() == "f70a6af77babf11afa3a50f7a15e532aa7baf3f3", "type of answer0_1 is not str. answer0_1 should be an str"
assert sha1(str(len(answer0_1)).encode("utf-8")+b"59e8").hexdigest() == "3b3b8f9d183db109adfa13225d741063b08dff85", "length of answer0_1 is not correct"
assert sha1(str(answer0_1.lower()).encode("utf-8")+b"59e8").hexdigest() == "6edfdac10655fefa98b28f34c9d1f860dcce8ff7", "value of answer0_1 is not correct"
assert sha1(str(answer0_1).encode("utf-8")+b"59e8").hexdigest() == "9dd39830e122b8184b1799ac7d2e687f95a317b7", "correct string value of answer0_1 but incorrect case of letters"

print('Success!')

## 1. Fruit Data Example 

In the agricultural industry, cleaning, sorting, grading, and packaging food products are all necessary tasks in the post-harvest process. Products are classified based on appearance, size and shape, attributes which helps determine the quality of the food. Sorting can be done by humans, but it is tedious and time consuming. Automatic sorting could help save time and money. Images of the food products are captured and analysed to determine visual characteristics. 

The [dataset](https://www.kaggle.com/mjamilmoughal/k-nearest-neighbor-classifier-to-predict-fruits/notebook) contains observations of fruit described with four features: (1) mass (in g), (2) width (in cm), (3) height (in cm), and (4) color score (on a scale from 0 - 1).

**Question 1.0** 
<br> {points: 1}

Load the file, `fruit_data.csv`, into your notebook. 

*Assign your data to an object called `fruit_data`.*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_data is None)).encode("utf-8")+b"a112").hexdigest() == "b557b9382e817d27247e595a35b0d3a83ac3ec88", "type of fruit_data is None is not bool. fruit_data is None should be a bool"
assert sha1(str(fruit_data is None).encode("utf-8")+b"a112").hexdigest() == "708f934d97e92ccaa033511e32a1424acd01a28a", "boolean value of fruit_data is None is not correct"

assert sha1(str(type(fruit_data.shape)).encode("utf-8")+b"a113").hexdigest() == "e5f99f9a3ba97caf55dc0f0343d9f98d5aeaafac", "type of fruit_data.shape is not tuple. fruit_data.shape should be a tuple"
assert sha1(str(len(fruit_data.shape)).encode("utf-8")+b"a113").hexdigest() == "4a6b8cf1cf2cb68fe8961d8e0fd7848d80a6bd3a", "length of fruit_data.shape is not correct"
assert sha1(str(sorted(map(str, fruit_data.shape))).encode("utf-8")+b"a113").hexdigest() == "d758d98096a08761529d799df7323bee3b5dec33", "values of fruit_data.shape are not correct"
assert sha1(str(fruit_data.shape).encode("utf-8")+b"a113").hexdigest() == "506310def3e283a6b2952f7e8eb2aa434e3465ac", "order of elements of fruit_data.shape is not correct"

assert sha1(str(type(fruit_data.fruit_name.dtype)).encode("utf-8")+b"a114").hexdigest() == "9c9d1156d3bd3ac97ec6785733d2b3f0e34e430d", "type of fruit_data.fruit_name.dtype is not correct"
assert sha1(str(fruit_data.fruit_name.dtype).encode("utf-8")+b"a114").hexdigest() == "64b5b851b58475f83ce9a6f208fdd4cfe4398537", "value of fruit_data.fruit_name.dtype is not correct"

assert sha1(str(type(fruit_data.fruit_name.unique())).encode("utf-8")+b"a115").hexdigest() == "707f09ffce8ac9c3c19ee6a28c808caba0cd8031", "type of fruit_data.fruit_name.unique() is not correct"
assert sha1(str(fruit_data.fruit_name.unique()).encode("utf-8")+b"a115").hexdigest() == "c366dcdda8d550b55e5eef2497ad4312c1a554ad", "value of fruit_data.fruit_name.unique() is not correct"

assert sha1(str(type(fruit_data.mass.values)).encode("utf-8")+b"a116").hexdigest() == "16497567ff3b43ac6f5883bcfff4afa7d6af6e5a", "type of fruit_data.mass.values is not correct"
assert sha1(str(fruit_data.mass.values).encode("utf-8")+b"a116").hexdigest() == "a717771cda8e463776d6bd2f12ff93a42f312a5b", "value of fruit_data.mass.values is not correct"

print('Success!')

Let's take a look at the first few observations in the fruit dataset. Run the cell below.

In [ ]:
# Run this cell.
fruit_data.head()

**Question 1.0.1** Multiple Choice:
<br> {points: 1}

**Which of the columns should we treat as categorical variables?**

A. Fruit label, width, fruit subtype

B. Fruit name, color score, height

C. Fruit label, fruit subtype, fruit name

D. Color score, mass, width 

*Assign your answer to an object called `answer1_0_1`. Make sure the correct answer is an uppercase letter. Remember to surround your answer with quotation marks (e.g. `"E"`).*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer1_0_1)).encode("utf-8")+b"a6e0c").hexdigest() == "78ad8e973a10e6225203a232c13b51e07410e0c7", "type of answer1_0_1 is not str. answer1_0_1 should be an str"
assert sha1(str(len(answer1_0_1)).encode("utf-8")+b"a6e0c").hexdigest() == "c928c37c29a1db11f3465bef596992606556af4a", "length of answer1_0_1 is not correct"
assert sha1(str(answer1_0_1.lower()).encode("utf-8")+b"a6e0c").hexdigest() == "ec11ea56291dfaddb9428e1344ca005c464b5637", "value of answer1_0_1 is not correct"
assert sha1(str(answer1_0_1).encode("utf-8")+b"a6e0c").hexdigest() == "d223cdb1aadd8c752ede1b214a1478585fbd1c4c", "correct string value of answer1_0_1 but incorrect case of letters"

print('Success!')

Run the cell below, and visually inspect the scatterplot to find the nearest neighbour to the first observation based on mass and width (the first observation has been circled for you).

In [ ]:
# Create the scatterplot
fruit_chart = alt.Chart(fruit_data).mark_point(size=15).encode(
    x=alt.X("mass").title("Mass (grams)"),
    y=alt.Y("width")
        .title("Width (cm)")
        .scale(zero=False),
    color=alt.Color("fruit_name").title("Fruit")
)
# Create the circle around point1
point1 = alt.Chart().mark_point(size=300, color='black').encode(
    x=alt.datum(fruit_data.loc[0, "mass"]),
    y=alt.datum(fruit_data.loc[0, "width"]),
    text=alt.datum('1')
)

# Layer the scatterplot, point1 circle, and a text annotation
fruit_chart + point1 + point1.mark_text(size=14, dx=12, align='left')

**Question 1.1** Multiple Choice: 
<br> {points: 1}

Based on the graph generated, what is the `fruit_name` of the closest data point to the one circled?

A. apple

B. lemon

C. mandarin 

D. orange

*Assign your answer to an object called `answer1_1`. Make sure the correct answer is an uppercase letter. Surround your answer with quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer1_1)).encode("utf-8")+b"adc95").hexdigest() == "f65cd5d5c9ff679b669dcd967b97b3b8744ef959", "type of answer1_1 is not str. answer1_1 should be an str"
assert sha1(str(len(answer1_1)).encode("utf-8")+b"adc95").hexdigest() == "6d7c58889cee9ce413e8290664d8cd9f3276bee3", "length of answer1_1 is not correct"
assert sha1(str(answer1_1.lower()).encode("utf-8")+b"adc95").hexdigest() == "9687d8e299d3616da92d06386aa467b1fe6af8d2", "value of answer1_1 is not correct"
assert sha1(str(answer1_1).encode("utf-8")+b"adc95").hexdigest() == "63b04d0a59a970ba2547aaf90b67cb154064bd6c", "correct string value of answer1_1 but incorrect case of letters"

print('Success!')

**Question 1.2**
<br> {points: 1}

Using mass and width, calculate the distance between the first observation and the second observation with the `euclidean_distances` function. 

We provide a scaffolding to get you started. 

*Assign your answer to an object called `fruit_dist_2`.*

In [ ]:
# ___ = euclidean_distances(fruit_data.loc[[___, 1], ["mass", ___]])

# your code here
raise NotImplementedError
fruit_dist_2

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_dist_2)).encode("utf-8")+b"51f4f").hexdigest() == "1002e3dba5717b2bed812c2e71539dff76591510", "type of fruit_dist_2 is not correct"
assert sha1(str(fruit_dist_2).encode("utf-8")+b"51f4f").hexdigest() == "515e07f951ee031fa4f2170f86c6bec9bed22867", "value of fruit_dist_2 is not correct"

print('Success!')

**Question 1.3**
<br> {points: 1}

Calculate the distance between the first and the the 44th observation in the fruit dataset using the mass and width variables. 

*Hint: remember that in Python, index starts from 0, so the 44th observation in Pandas Dataframe corresponds to index 43*

*Assign your answer to an object called `fruit_dist_44`.*

In [ ]:
# your code here
raise NotImplementedError
fruit_dist_44

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_dist_44)).encode("utf-8")+b"14e1b").hexdigest() == "3d892654daebf107b61197ecae5d0450aa54b42f", "type of fruit_dist_44 is not correct"
assert sha1(str(fruit_dist_44).encode("utf-8")+b"14e1b").hexdigest() == "49b24683a0230c884cf882b96bddae858308dde5", "value of fruit_dist_44 is not correct"

print('Success!')

Let's circle these three observations on the chart we created earlier.

In [ ]:
# When we want to annotate multiple points it is efficient to create a separate data frame
# and a single chart instead of one chart per point
points = alt.Chart(
    fruit_data.loc[[0, 1, 43], ['mass', 'width']].assign(point_number=[1, 2, 44])
).mark_point(size=300, color='black').encode(
    x='mass',
    y='width',
    text='point_number'
)

fruit_chart + points + points.mark_text(size=14, dx=12, align='left')

What do you notice about your answers from **Question 1.2 & 1.3** that you just calculated? Is it what you would expect given the scatter plot above? Why or why not? Discuss with your neighbour. 

*Hint: Look at where the observations are on the scatterplot in the cell above this question, and what might happen if we changed grams into kilograms to measure the mass?*


**Question 1.4** Multiple Choice:
<br> {points: 1}

The distance between the first and second observation is 12.01 and the distance between the first and 44th observation is 2.33. By the formula, observation 1 and 44 are closer, however, if we look at the scatterplot the distance of the first observation to the second observation appears closer than to the 44th observation. 

Which of the following statements is correct?

A. A difference of 12 g in mass between observation 1 and 2 is large compared to a difference of 1.2 cm in width between observation 1 and 44. Consequently, mass will drive the classification results, and width will have less of an effect. 

B. If we measured mass in kilograms, then we’d get different nearest neighbours.

C. We should standardize the data so that all variables will be on a comparable scale. 

D. All of the above. 

*Assign your answer to an object called `answer1_4`. Make sure the correct answer is an uppercase letter. Surround your answer with quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer1_4)).encode("utf-8")+b"ef865").hexdigest() == "f9bec660245450950ea78e5df29a898814ed4e7e", "type of answer1_4 is not str. answer1_4 should be an str"
assert sha1(str(len(answer1_4)).encode("utf-8")+b"ef865").hexdigest() == "b752766645743c09fca27938c0f49419ab25123e", "length of answer1_4 is not correct"
assert sha1(str(answer1_4.lower()).encode("utf-8")+b"ef865").hexdigest() == "fdce6f58a4a6bc3489623863420da425460ccf64", "value of answer1_4 is not correct"
assert sha1(str(answer1_4).encode("utf-8")+b"ef865").hexdigest() == "bc550636f1a022814a914a80ce95aa6cac8a87b1", "correct string value of answer1_4 but incorrect case of letters"

print('Success!')

**Question 1.5**
<br> {points: 1}

Let's create a `preprocessor` to *standardize* (i.e., center and scale) all of the variables in the fruit dataset. Centering will make sure that every variable has an average of 0, and scaling will make sure that every variable has standard deviation of 1. We will use the `StandardScaler` in the `preprocessor`. Then `fit_transform` the preprocessor so that we can examine the output.

Fit and transform your preprocessor with predictors `mass`, `width`, `height`, and `color_score`. Pass through all the remaining columns.

Name the preprocessor `fruit_data_preprocessor`, and name the preprocessed data frame `fruit_data_scaled`.

In [ ]:
# ___ = ___(
#     (StandardScaler(), [___, ___, ___, ___]),
#     remainder=___
# )

# ___ = fruit_data_preprocessor.___(___)


# your code here
raise NotImplementedError
fruit_data_scaled.head()

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_data_scaled is None)).encode("utf-8")+b"f25fe").hexdigest() == "4eca96cbba14d6870b18ada76a5fa5a7f8657f44", "type of fruit_data_scaled is None is not bool. fruit_data_scaled is None should be a bool"
assert sha1(str(fruit_data_scaled is None).encode("utf-8")+b"f25fe").hexdigest() == "eb5d5311a8b0fe60645f283fcb1bd89bb50155a9", "boolean value of fruit_data_scaled is None is not correct"

assert sha1(str(type(fruit_data_scaled.shape)).encode("utf-8")+b"f25ff").hexdigest() == "1d80f3888cf7d3b5bdc8aa60c18336bf4ae02ff4", "type of fruit_data_scaled.shape is not tuple. fruit_data_scaled.shape should be a tuple"
assert sha1(str(len(fruit_data_scaled.shape)).encode("utf-8")+b"f25ff").hexdigest() == "aea322dbb0dc7cc099f35d9e2fc1f5acaef7ca21", "length of fruit_data_scaled.shape is not correct"
assert sha1(str(sorted(map(str, fruit_data_scaled.shape))).encode("utf-8")+b"f25ff").hexdigest() == "88e001a787391e2f87aaa708d163603e8242d2d1", "values of fruit_data_scaled.shape are not correct"
assert sha1(str(fruit_data_scaled.shape).encode("utf-8")+b"f25ff").hexdigest() == "9637d0fae90893f317f1dc3ec57e1501436c6093", "order of elements of fruit_data_scaled.shape is not correct"

assert sha1(str(type(fruit_data_scaled.fruit_name.dtype)).encode("utf-8")+b"f2600").hexdigest() == "d9d7cfa27226c7a0e00489369dd26cc14d83c995", "type of fruit_data_scaled.fruit_name.dtype is not correct"
assert sha1(str(fruit_data_scaled.fruit_name.dtype).encode("utf-8")+b"f2600").hexdigest() == "e0a3a46fc6c83e9355fa51ba62d4a2e6acbf514b", "value of fruit_data_scaled.fruit_name.dtype is not correct"

assert sha1(str(type(np.mean(fruit_data_scaled.mass.dropna()))).encode("utf-8")+b"f2601").hexdigest() == "89b59446a9320a83a4a1556cc8f09c6d1b57c11b", "type of np.mean(fruit_data_scaled.mass.dropna()) is not correct"
assert sha1(str(np.mean(fruit_data_scaled.mass.dropna())).encode("utf-8")+b"f2601").hexdigest() == "ce9e59e0cf9ba142f75d51178846d2c1d26026e7", "value of np.mean(fruit_data_scaled.mass.dropna()) is not correct"

assert sha1(str(type(np.mean(fruit_data_scaled.height.dropna()))).encode("utf-8")+b"f2602").hexdigest() == "393de24af4ce830bc129e9143d46ee528c7b8b13", "type of np.mean(fruit_data_scaled.height.dropna()) is not correct"
assert sha1(str(np.mean(fruit_data_scaled.height.dropna())).encode("utf-8")+b"f2602").hexdigest() == "fa7c3849dc153e931658b1bd7efb766ccc1b86ba", "value of np.mean(fruit_data_scaled.height.dropna()) is not correct"

assert sha1(str(type(np.mean(fruit_data_scaled.width.dropna()))).encode("utf-8")+b"f2603").hexdigest() == "d00b9d1eea7853908bbcf8268289d02e1e2445c8", "type of np.mean(fruit_data_scaled.width.dropna()) is not correct"
assert sha1(str(np.mean(fruit_data_scaled.width.dropna())).encode("utf-8")+b"f2603").hexdigest() == "c939f57d7fd52a8eaa55567d9d1d8887c527cb16", "value of np.mean(fruit_data_scaled.width.dropna()) is not correct"

assert sha1(str(type(np.mean(fruit_data_scaled.color_score.dropna()))).encode("utf-8")+b"f2604").hexdigest() == "af165a9cd512364cf2569ad636f2aac3085754b9", "type of np.mean(fruit_data_scaled.color_score.dropna()) is not correct"
assert sha1(str(np.mean(fruit_data_scaled.color_score.dropna())).encode("utf-8")+b"f2604").hexdigest() == "ff3465e7d0be833cbfae4cc08f00d010546a3337", "value of np.mean(fruit_data_scaled.color_score.dropna()) is not correct"

assert sha1(str(type(np.std(fruit_data_scaled.mass.dropna()))).encode("utf-8")+b"f2605").hexdigest() == "e5471f8576452c679343e16eea1e1720fc08cdee", "type of np.std(fruit_data_scaled.mass.dropna()) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(np.std(fruit_data_scaled.mass.dropna()), 2)).encode("utf-8")+b"f2605").hexdigest() == "6c82d4566c488c0ddb5a91eee728b8b67b968878", "value of np.std(fruit_data_scaled.mass.dropna()) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(np.std(fruit_data_scaled.height.dropna()))).encode("utf-8")+b"f2606").hexdigest() == "08c8cab95228124af8044befed88769a2989113f", "type of np.std(fruit_data_scaled.height.dropna()) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(np.std(fruit_data_scaled.height.dropna()), 2)).encode("utf-8")+b"f2606").hexdigest() == "607a9f3f56397d1638ddcd6965783d54f7fafe23", "value of np.std(fruit_data_scaled.height.dropna()) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(np.std(fruit_data_scaled.width.dropna()))).encode("utf-8")+b"f2607").hexdigest() == "1d1412f951409a4b4e9ab5ae86688a2e70469ed9", "type of np.std(fruit_data_scaled.width.dropna()) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(np.std(fruit_data_scaled.width.dropna()), 2)).encode("utf-8")+b"f2607").hexdigest() == "e3cc7bf0b44f2ba90260474bc53e6b7b408d8d12", "value of np.std(fruit_data_scaled.width.dropna()) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(np.std(fruit_data_scaled.color_score.dropna()))).encode("utf-8")+b"f2608").hexdigest() == "e0fedb6f6b2c02a9c669785f5317c77cee756672", "type of np.std(fruit_data_scaled.color_score.dropna()) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(np.std(fruit_data_scaled.color_score.dropna()), 2)).encode("utf-8")+b"f2608").hexdigest() == "1e2dc78abca79bd7ea9d2fe09ca96d4cbadca652", "value of np.std(fruit_data_scaled.color_score.dropna()) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(fruit_data_preprocessor is None)).encode("utf-8")+b"f2609").hexdigest() == "1051f3a0eb8db2c0420580c2ceb210d87d6fc4f7", "type of fruit_data_preprocessor is None is not bool. fruit_data_preprocessor is None should be a bool"
assert sha1(str(fruit_data_preprocessor is None).encode("utf-8")+b"f2609").hexdigest() == "9ad5efb0300719e94652931211b98998a0e6ad41", "boolean value of fruit_data_preprocessor is None is not correct"

assert sha1(str(type(fruit_data_preprocessor.transformers_[1][2])).encode("utf-8")+b"f260a").hexdigest() == "f3b1658a30e341f1386b8a229711473eb27b976e", "type of fruit_data_preprocessor.transformers_[1][2] is not list. fruit_data_preprocessor.transformers_[1][2] should be a list"
assert sha1(str(len(fruit_data_preprocessor.transformers_[1][2])).encode("utf-8")+b"f260a").hexdigest() == "61be071e345d1d11d793c50a3a1213c42215d18f", "length of fruit_data_preprocessor.transformers_[1][2] is not correct"
assert sha1(str(sorted(map(str, fruit_data_preprocessor.transformers_[1][2]))).encode("utf-8")+b"f260a").hexdigest() == "02889e6d896cadba7410704feac2cc930c06dfe3", "values of fruit_data_preprocessor.transformers_[1][2] are not correct"
assert sha1(str(fruit_data_preprocessor.transformers_[1][2]).encode("utf-8")+b"f260a").hexdigest() == "89b1c87c6bbf8bef32b5498a3e885aa052dac9e4", "order of elements of fruit_data_preprocessor.transformers_[1][2] is not correct"

print('Success!')

**Question 1.6**
<br> {points: 1}

Let's repeat **Question 1.2 and 1.3** with the scaled variables:

- calculate the distance with the scaled mass and width variables between observations 1 and 2
- calculate the distances with the scaled mass and width variables between observations 1 and 44 

After you do this, think about how these distances compared to the distances you computed in **Question 1.2 and 1.3** for the same points.

*Assign your answers to objects called `distance_2` and `distance_44` respectively.*

In [ ]:
# your code here
raise NotImplementedError
print(distance_2)
print(distance_44)

In [ ]:
from hashlib import sha1
assert sha1(str(type(distance_2 is None)).encode("utf-8")+b"663ce").hexdigest() == "7e130f41ce84663de8eb1352a2ddbafe99d72d04", "type of distance_2 is None is not bool. distance_2 is None should be a bool"
assert sha1(str(distance_2 is None).encode("utf-8")+b"663ce").hexdigest() == "fa64ca24799595eb2345847767542d041659498a", "boolean value of distance_2 is None is not correct"

assert sha1(str(type(distance_44 is None)).encode("utf-8")+b"663cf").hexdigest() == "0a962e013c6793cc088e690751b2a7b3ab98c77d", "type of distance_44 is None is not bool. distance_44 is None should be a bool"
assert sha1(str(distance_44 is None).encode("utf-8")+b"663cf").hexdigest() == "8fdce830ac1ae69e7705200316f9dc6d36e9dd61", "boolean value of distance_44 is None is not correct"

assert sha1(str(type(distance_2)).encode("utf-8")+b"663d0").hexdigest() == "b4941db682e95f667a5fa0e95795638d06333a01", "type of type(distance_2) is not correct"

assert sha1(str(type(distance_44)).encode("utf-8")+b"663d1").hexdigest() == "4c68f8d25ab18c0c629af3568392c3aa8024b51a", "type of type(distance_44) is not correct"

assert sha1(str(type(distance_2)).encode("utf-8")+b"663d2").hexdigest() == "76baa32945e5cd92c941501ff34cc38a0d1e8068", "type of distance_2 is not correct"
assert sha1(str(distance_2).encode("utf-8")+b"663d2").hexdigest() == "d3a9f0a5e87b47bf8d54fb49d1dddc0a60211ce6", "value of distance_2 is not correct"

assert sha1(str(type(distance_44)).encode("utf-8")+b"663d3").hexdigest() == "85f5b8ff0ced78af224bc123cd98dfb1ebbbff44", "type of distance_44 is not correct"
assert sha1(str(distance_44).encode("utf-8")+b"663d3").hexdigest() == "5852e911331b5f8f08a91e8b4f5214cb6e596516", "value of distance_44 is not correct"

print('Success!')

**Question 1.7**
<br> {points: 1}

Make a scatterplot of scaled mass on the horizontal axis and scaled color score on the vertical axis. Color the points by fruit name. 

*Assign your plot to an object called `fruit_plot`. Make sure to do all the things to make an effective visualization.*

In [ ]:
# your code here
raise NotImplementedError
fruit_plot

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_plot is None)).encode("utf-8")+b"8a68b").hexdigest() == "2eb88fd37a2f9ad35c86baea93a7c47a34733fc9", "type of fruit_plot is None is not bool. fruit_plot is None should be a bool"
assert sha1(str(fruit_plot is None).encode("utf-8")+b"8a68b").hexdigest() == "96559b06b6b1702b0d9e1e2c8d8c251cbbd9c0d8", "boolean value of fruit_plot is None is not correct"

assert sha1(str(type(fruit_plot.encoding.x['shorthand'])).encode("utf-8")+b"8a68c").hexdigest() == "990fdef163dcedc13a931eba5fdc5a880b23f71f", "type of fruit_plot.encoding.x['shorthand'] is not str. fruit_plot.encoding.x['shorthand'] should be an str"
assert sha1(str(len(fruit_plot.encoding.x['shorthand'])).encode("utf-8")+b"8a68c").hexdigest() == "0374e9f8a14be83fa1e1c62df034a71203996d53", "length of fruit_plot.encoding.x['shorthand'] is not correct"
assert sha1(str(fruit_plot.encoding.x['shorthand'].lower()).encode("utf-8")+b"8a68c").hexdigest() == "dfef3739fefa652a9b39e2fe8e35725607356a66", "value of fruit_plot.encoding.x['shorthand'] is not correct"
assert sha1(str(fruit_plot.encoding.x['shorthand']).encode("utf-8")+b"8a68c").hexdigest() == "dfef3739fefa652a9b39e2fe8e35725607356a66", "correct string value of fruit_plot.encoding.x['shorthand'] but incorrect case of letters"

assert sha1(str(type(fruit_plot.encoding.y['shorthand'])).encode("utf-8")+b"8a68d").hexdigest() == "169bb4f2d655079959d4662419dbcaec59ec6167", "type of fruit_plot.encoding.y['shorthand'] is not str. fruit_plot.encoding.y['shorthand'] should be an str"
assert sha1(str(len(fruit_plot.encoding.y['shorthand'])).encode("utf-8")+b"8a68d").hexdigest() == "42b25752bc14f8efaf0a98a2cd836990d47e5111", "length of fruit_plot.encoding.y['shorthand'] is not correct"
assert sha1(str(fruit_plot.encoding.y['shorthand'].lower()).encode("utf-8")+b"8a68d").hexdigest() == "5f08604168bab219696386b6df81af2ec5aed4a1", "value of fruit_plot.encoding.y['shorthand'] is not correct"
assert sha1(str(fruit_plot.encoding.y['shorthand']).encode("utf-8")+b"8a68d").hexdigest() == "5f08604168bab219696386b6df81af2ec5aed4a1", "correct string value of fruit_plot.encoding.y['shorthand'] but incorrect case of letters"

assert sha1(str(type(fruit_plot.encoding.color['shorthand'])).encode("utf-8")+b"8a68e").hexdigest() == "73ce6be309d9286e646a0380fcab4c00aa65bd01", "type of fruit_plot.encoding.color['shorthand'] is not str. fruit_plot.encoding.color['shorthand'] should be an str"
assert sha1(str(len(fruit_plot.encoding.color['shorthand'])).encode("utf-8")+b"8a68e").hexdigest() == "17a2a30b7b2a42ba128526f262ace7dbf03f6b22", "length of fruit_plot.encoding.color['shorthand'] is not correct"
assert sha1(str(fruit_plot.encoding.color['shorthand'].lower()).encode("utf-8")+b"8a68e").hexdigest() == "76d2b478d1ffb793a29cb9239924296395d3bb7c", "value of fruit_plot.encoding.color['shorthand'] is not correct"
assert sha1(str(fruit_plot.encoding.color['shorthand']).encode("utf-8")+b"8a68e").hexdigest() == "76d2b478d1ffb793a29cb9239924296395d3bb7c", "correct string value of fruit_plot.encoding.color['shorthand'] but incorrect case of letters"

assert sha1(str(type(fruit_plot.mark)).encode("utf-8")+b"8a68f").hexdigest() == "76d38afe65402afe2de3222397ceb0231d524f26", "type of fruit_plot.mark is not str. fruit_plot.mark should be an str"
assert sha1(str(len(fruit_plot.mark)).encode("utf-8")+b"8a68f").hexdigest() == "c66527296d6886c31dc1ffcf6efdbd56af9a433b", "length of fruit_plot.mark is not correct"
assert sha1(str(fruit_plot.mark.lower()).encode("utf-8")+b"8a68f").hexdigest() == "c7941e4e9f11393086be5f804d31d0d4e59cc05c", "value of fruit_plot.mark is not correct"
assert sha1(str(fruit_plot.mark).encode("utf-8")+b"8a68f").hexdigest() == "c7941e4e9f11393086be5f804d31d0d4e59cc05c", "correct string value of fruit_plot.mark but incorrect case of letters"

assert sha1(str(type(isinstance(fruit_plot.encoding.x['title'], str))).encode("utf-8")+b"8a690").hexdigest() == "0a4f1696bfd421d1ccecb2458a975c4e7786e31d", "type of isinstance(fruit_plot.encoding.x['title'], str) is not bool. isinstance(fruit_plot.encoding.x['title'], str) should be a bool"
assert sha1(str(isinstance(fruit_plot.encoding.x['title'], str)).encode("utf-8")+b"8a690").hexdigest() == "06f18fbda4027c9bb364f9839311456225184150", "boolean value of isinstance(fruit_plot.encoding.x['title'], str) is not correct"

assert sha1(str(type(isinstance(fruit_plot.encoding.y['title'], str))).encode("utf-8")+b"8a691").hexdigest() == "473909fcf26823364d666d1009575523e1c15bef", "type of isinstance(fruit_plot.encoding.y['title'], str) is not bool. isinstance(fruit_plot.encoding.y['title'], str) should be a bool"
assert sha1(str(isinstance(fruit_plot.encoding.y['title'], str)).encode("utf-8")+b"8a691").hexdigest() == "57a44c8d3b9b8a8bff8e69f6018e46ce89c8eb9b", "boolean value of isinstance(fruit_plot.encoding.y['title'], str) is not correct"

assert sha1(str(type(isinstance(fruit_plot.encoding.color['title'], str))).encode("utf-8")+b"8a692").hexdigest() == "546fcb8f3a69f38cab39fa96e8dbcd1de01be4aa", "type of isinstance(fruit_plot.encoding.color['title'], str) is not bool. isinstance(fruit_plot.encoding.color['title'], str) should be a bool"
assert sha1(str(isinstance(fruit_plot.encoding.color['title'], str)).encode("utf-8")+b"8a692").hexdigest() == "215413dbe7a8ccf663d36f848f79313577dbacda", "boolean value of isinstance(fruit_plot.encoding.color['title'], str) is not correct"

print('Success!')

**Question 1.8** 
<br> {points: 3}

Suppose we have a new observation in the fruit dataset with scaled mass 0.5 and scaled color score 0.5.

Just by looking at the scatterplot, how would you classify this observation using K-nearest neighbours if you use K = 3? Explain how you arrived at your answer.

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

**Question 1.9**
<br> {points: 1}

Now, let's use the `scikit-learn` package to predict `fruit_name` for another new observation. First, create a K-nearest neighbour model with $K=5$ neighbors. Name this model specification `knn_spec`.

Then create a new preprocessor named `fruit_data_preprocessor_2` that centers and scales the predictors, but only uses `mass` and `color_score` as predictors. We can drop all other unused columns. Name the predictor as `X` and the target `y`.

Combine the preprocessor with your neighbour model `knn_spec` in a pipeline, and fit to the `fruit_data` dataset. 

*Name the fitted model `fruit_fit`.*

In [ ]:
# ___ = KNeighborsClassifier(n_neighbors=___)

# ____ = make_column_transformer(
#     (___, [___, ___]),
# )

# X = ____.drop(columns=[___])
# y = ___[___]

# ___ = ___(___, ___).fit(___, ___)

# your code here
raise NotImplementedError
fruit_fit

In [ ]:
from hashlib import sha1
assert sha1(str(type(knn_spec is None)).encode("utf-8")+b"2de2e").hexdigest() == "2f656698603296a3c30fd64bc9b2c4557f8b4879", "type of knn_spec is None is not bool. knn_spec is None should be a bool"
assert sha1(str(knn_spec is None).encode("utf-8")+b"2de2e").hexdigest() == "ff1547749fefe8344039793a7db80de2ed86f8d1", "boolean value of knn_spec is None is not correct"

assert sha1(str(type(knn_spec.n_neighbors)).encode("utf-8")+b"2de2f").hexdigest() == "54476aa53aea2f8fe3d6caec0f2c94118b7ab982", "type of knn_spec.n_neighbors is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(knn_spec.n_neighbors).encode("utf-8")+b"2de2f").hexdigest() == "7d812171af345a6d2547b75d454ffbfb2ad1cfe8", "value of knn_spec.n_neighbors is not correct"

assert sha1(str(type(knn_spec.effective_metric_)).encode("utf-8")+b"2de30").hexdigest() == "b2a92f208e4d7ffdfe0fe538ebaae84569b80fe0", "type of knn_spec.effective_metric_ is not str. knn_spec.effective_metric_ should be an str"
assert sha1(str(len(knn_spec.effective_metric_)).encode("utf-8")+b"2de30").hexdigest() == "f28fe5343cceaa19ae730e78b38337ebcab525cb", "length of knn_spec.effective_metric_ is not correct"
assert sha1(str(knn_spec.effective_metric_.lower()).encode("utf-8")+b"2de30").hexdigest() == "32e17aab20b482930453d26d4c28f002db625560", "value of knn_spec.effective_metric_ is not correct"
assert sha1(str(knn_spec.effective_metric_).encode("utf-8")+b"2de30").hexdigest() == "32e17aab20b482930453d26d4c28f002db625560", "correct string value of knn_spec.effective_metric_ but incorrect case of letters"

assert sha1(str(type(fruit_data_preprocessor_2 is None)).encode("utf-8")+b"2de31").hexdigest() == "9b4c6d44c83e723fc72e11ade4336f7caab31399", "type of fruit_data_preprocessor_2 is None is not bool. fruit_data_preprocessor_2 is None should be a bool"
assert sha1(str(fruit_data_preprocessor_2 is None).encode("utf-8")+b"2de31").hexdigest() == "f48cf9c4e505fba10dc03daaf32c9eb3dcdb6515", "boolean value of fruit_data_preprocessor_2 is None is not correct"

assert sha1(str(type(fruit_data_preprocessor_2.transformers_[0][2])).encode("utf-8")+b"2de32").hexdigest() == "1a692d250d78748e9ae32b88a817982543507611", "type of fruit_data_preprocessor_2.transformers_[0][2] is not list. fruit_data_preprocessor_2.transformers_[0][2] should be a list"
assert sha1(str(len(fruit_data_preprocessor_2.transformers_[0][2])).encode("utf-8")+b"2de32").hexdigest() == "d59bc27c28012e71dc180e5197a586765e19c9bc", "length of fruit_data_preprocessor_2.transformers_[0][2] is not correct"
assert sha1(str(sorted(map(str, fruit_data_preprocessor_2.transformers_[0][2]))).encode("utf-8")+b"2de32").hexdigest() == "faa57b3393d92de7cd9dfe8e565ecd7099393e60", "values of fruit_data_preprocessor_2.transformers_[0][2] are not correct"
assert sha1(str(fruit_data_preprocessor_2.transformers_[0][2]).encode("utf-8")+b"2de32").hexdigest() == "b1b3246e35d97d514d503d0423224a472ff88205", "order of elements of fruit_data_preprocessor_2.transformers_[0][2] is not correct"

assert sha1(str(type(fruit_fit is None)).encode("utf-8")+b"2de33").hexdigest() == "3c91488616850895d26bc585c4564e2470a9525d", "type of fruit_fit is None is not bool. fruit_fit is None should be a bool"
assert sha1(str(fruit_fit is None).encode("utf-8")+b"2de33").hexdigest() == "a43950ee20d197dd929e3fe15963d9a85cb7250a", "boolean value of fruit_fit is None is not correct"

assert sha1(str(type(type(fruit_fit))).encode("utf-8")+b"2de34").hexdigest() == "fba3e1eae86f1348b80b1e2b220aab7692d1e014", "type of type(fruit_fit) is not correct"
assert sha1(str(type(fruit_fit)).encode("utf-8")+b"2de34").hexdigest() == "09b8692c89f3acf882c802f89e1fd50958e5f8cd", "value of type(fruit_fit) is not correct"

assert sha1(str(type(fruit_fit.named_steps.kneighborsclassifier.n_neighbors)).encode("utf-8")+b"2de35").hexdigest() == "0550e0522094a23b7ae4ca9e4c2ec6358d93c920", "type of fruit_fit.named_steps.kneighborsclassifier.n_neighbors is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(fruit_fit.named_steps.kneighborsclassifier.n_neighbors).encode("utf-8")+b"2de35").hexdigest() == "7c417f3eafdc436b9d08d1d4d3dad67e63db2f50", "value of fruit_fit.named_steps.kneighborsclassifier.n_neighbors is not correct"

assert sha1(str(type(fruit_fit.named_steps.kneighborsclassifier.effective_metric_)).encode("utf-8")+b"2de36").hexdigest() == "b279e909d7321a30f7b90755f4f44bf384a66946", "type of fruit_fit.named_steps.kneighborsclassifier.effective_metric_ is not str. fruit_fit.named_steps.kneighborsclassifier.effective_metric_ should be an str"
assert sha1(str(len(fruit_fit.named_steps.kneighborsclassifier.effective_metric_)).encode("utf-8")+b"2de36").hexdigest() == "51328a92f6bf057e32a295c29f8af2f796835377", "length of fruit_fit.named_steps.kneighborsclassifier.effective_metric_ is not correct"
assert sha1(str(fruit_fit.named_steps.kneighborsclassifier.effective_metric_.lower()).encode("utf-8")+b"2de36").hexdigest() == "a10ad5a701b9778b5269fb0583fdf5efc0038069", "value of fruit_fit.named_steps.kneighborsclassifier.effective_metric_ is not correct"
assert sha1(str(fruit_fit.named_steps.kneighborsclassifier.effective_metric_).encode("utf-8")+b"2de36").hexdigest() == "a10ad5a701b9778b5269fb0583fdf5efc0038069", "correct string value of fruit_fit.named_steps.kneighborsclassifier.effective_metric_ but incorrect case of letters"

print('Success!')

**Question 1.10**
<br> {points: 1}

The new observation we are interested in has mass 150g and color score 0.73. Create a new dataframe `mass = 150` and `color_score = 0.73` and call it `new_fruit`. Then, pass `fruit_fit` and `new_fruit` to the `predict` function to predict the class for the new fruit observation. Save your prediction to an object named `fruit_predicted`.

In [ ]:
# your code here
raise NotImplementedError
fruit_predicted

In [ ]:
from hashlib import sha1
assert sha1(str(type(new_fruit is None)).encode("utf-8")+b"9d4b9").hexdigest() == "3222a4457ea7d947a5727ca89ab62fd52f729b74", "type of new_fruit is None is not bool. new_fruit is None should be a bool"
assert sha1(str(new_fruit is None).encode("utf-8")+b"9d4b9").hexdigest() == "ef575b67eb4b722e49513f4d0bedd172876555e1", "boolean value of new_fruit is None is not correct"

assert sha1(str(type(new_fruit.shape)).encode("utf-8")+b"9d4ba").hexdigest() == "04a8d89cbf1d3b168b1bac685df0fe6aa0f45181", "type of new_fruit.shape is not tuple. new_fruit.shape should be a tuple"
assert sha1(str(len(new_fruit.shape)).encode("utf-8")+b"9d4ba").hexdigest() == "52f1ed8364cf562f0b8b41f03321e26ea99bfc53", "length of new_fruit.shape is not correct"
assert sha1(str(sorted(map(str, new_fruit.shape))).encode("utf-8")+b"9d4ba").hexdigest() == "1a531e9de4e3b928d7fad21623aa618c22dd130e", "values of new_fruit.shape are not correct"
assert sha1(str(new_fruit.shape).encode("utf-8")+b"9d4ba").hexdigest() == "863d4302dbd1a19b33778c34d693597c0a5e896b", "order of elements of new_fruit.shape is not correct"

assert sha1(str(type(new_fruit.mass.values)).encode("utf-8")+b"9d4bb").hexdigest() == "7526d14bb84ec36bbeb125019dd8281dbf46be98", "type of new_fruit.mass.values is not correct"
assert sha1(str(new_fruit.mass.values).encode("utf-8")+b"9d4bb").hexdigest() == "57d625833c0f92a5967ca29580ddac60fc8d175c", "value of new_fruit.mass.values is not correct"

assert sha1(str(type(new_fruit.color_score.values)).encode("utf-8")+b"9d4bc").hexdigest() == "22136490cc7459081a0f956868a5517b6c95a81c", "type of new_fruit.color_score.values is not correct"
assert sha1(str(new_fruit.color_score.values).encode("utf-8")+b"9d4bc").hexdigest() == "4eeff2722fbb1c1aeb3eb28059a40c7257956950", "value of new_fruit.color_score.values is not correct"

assert sha1(str(type(fruit_predicted)).encode("utf-8")+b"9d4bd").hexdigest() == "c0fe127c3976037e0a96c59de49d4fd63796dd85", "type of fruit_predicted is not correct"
assert sha1(str(fruit_predicted).encode("utf-8")+b"9d4bd").hexdigest() == "a64d7e8039ad9f84b2534a5d4e29cf15d5254b28", "value of fruit_predicted is not correct"

print('Success!')

**Question 1.11** 
<br> {points: 3}

Revisiting `fruit_plot` and considering the prediction given by K-nearest neighbours above, do you think the classification model did a "good" job predicting? Could you have done/do better? Given what we know this far in the course, what might we want to do to help with tricky prediction cases such as this?

*You can use the code below to visualize the observation whose label we just tried to predict.*

In [ ]:
fruit_plot + (
    # Standardize the new data point with transformer fitted on the original data
    alt.Chart(fruit_data_preprocessor_2.transform(new_fruit))
    .mark_circle(size=80, color='black').encode(
        x="mass",
        y="color_score"
    )
)

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

**Question 1.12**
<br> {points: 1}

Now do K-nearest neighbours classification again with the same data set, same K, and same new observation. However, this time, let's use **all the columns in the dataset as predictors (except for the categorical `fruit_label` and `fruit_subtype` variables).** Therefore, you would need to make a new preprocessor.

We have provided the `new_fruit_all` dataframe below, which encodes the predictors for our new observation. Your job is to use K-nearest neighbours to predict the class of this point. You can reuse the model specification `knn_spec` that you created earlier. 

Name the new predictor as `X_2` and new target `y_2`.

*Assign your answer (the output of `predict`) to an object called `fruit_all_predicted`.*

In [ ]:
# This is the new observation to predict class label for
new_fruit_all = pd.DataFrame(
    [[150, 6, 10, 0.73]],
    columns=[
        "mass",
        "width",
        "height",
        "color_score",
    ],
)

# no hints this time!

# your code here
raise NotImplementedError
fruit_all_predicted

In [ ]:
from hashlib import sha1
assert sha1(str(type(fruit_all_predicted)).encode("utf-8")+b"1b68e").hexdigest() == "438b64ff5db5790eaf1b10b54d223db3e99d7ce7", "type of fruit_all_predicted is not correct"
assert sha1(str(fruit_all_predicted).encode("utf-8")+b"1b68e").hexdigest() == "f4716bb2f048786e58330db84126dad27a9c6268", "value of fruit_all_predicted is not correct"

print('Success!')

**Question 1.13** 
<br> {points: 3}

Did your second classification on the same data set with the same K change the prediction? If so, why do you think this happened?

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

## 2. Wheat Seed Dataset

X-ray images can be used to analyze and sort seeds. In [this data set](https://archive.ics.uci.edu/ml/datasets/seeds), we have 7 measurements from x-ray images from 3 varieties of wheat seeds (Kama, Rosa and Canadian). 

**Question 2.0**
<br> {points: 3}

Let's use `scikit-learn` to perform K-nearest neighbours to classify the wheat variety of seeds. The data set is available here: https://archive.ics.uci.edu/ml/machine-learning-databases/00236/seeds_dataset.txt. **Download the data set directly from this URL using the `pd.read_csv` function with `delimiter='\t'`**, which is helpful when the columns are separated by one or more white spaces.

The seven measurements were taken below for each wheat kernel:
1. area A, 
2. perimeter P, 
3. compactness C = 4*pi*A/P^2, 
4. length of kernel, 
5. width of kernel, 
6. asymmetry coefficient 
7. length of kernel groove. 

The last column in the data set is the variety label. The mapping for the numbers to varieties is listed below:

- 1 == Kama
- 2 == Rosa
- 3 == Canadian

Use `scikit-learn` with this data to perform K-nearest neighbours to classify the wheat variety of a new seed we measure with the given observed measurements (from an x-ray image) listed above. Specify that we want $K = 5$ neighbors to perform the classification. 

*Assign your answer to an object called `seed_predict`.*

Hints: 
- `names` can be used to specify the column names of a data frame.
- There are some nan values in the dataset, please use `dropna` to drop the nan values in the dataset before passing it into the K-nearest neighbours model.

In [ ]:
# This is the new observation to predict
new_seed = pd.DataFrame(
    [[12.1, 14.2, 0.9, 4.9, 2.8, 3.0, 5.1]],
    columns=[
        "area",
        "perimeter",
        "compactness",
        "length",
        "width",
        "asymmetry_coefficient",
        "groove_length",
    ],
)

# your code here
raise NotImplementedError
seed_predict

**Question 2.1** Multiple Choice:
<br> {points: 1}

What is classification of the `new_seed` observation?

A. Kama

B. Rosa

C. Canadian

*Assign your answer to an object called `answer2_1`. Make sure your answer is in uppercase and is surrounded by quotation marks (e.g. `"F"`).*


In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer2_1)).encode("utf-8")+b"3e453").hexdigest() == "15ef2db7623049d810320b160bdd9f8cb2aeb736", "type of answer2_1 is not str. answer2_1 should be an str"
assert sha1(str(len(answer2_1)).encode("utf-8")+b"3e453").hexdigest() == "3bcfa64d6526f62fac5d53628275e523dc0b74e2", "length of answer2_1 is not correct"
assert sha1(str(answer2_1.lower()).encode("utf-8")+b"3e453").hexdigest() == "8997ae32bba2a701404281b140fb334d0e4f12f6", "value of answer2_1 is not correct"
assert sha1(str(answer2_1).encode("utf-8")+b"3e453").hexdigest() == "c6bcb7785029af288b81e0add1238bcc7de9234a", "correct string value of answer2_1 but incorrect case of letters"

print('Success!')